In [7]:
# 1. Implementarea unei aplicatii de gestionare a unui abonament de streaming
# Avand implementarea clasei StreamingService, implementati:
# - o clasa BankAccount (owner: str, iban: str, balance: int default 0) cu functii specifice unui cont (deposit, withdraw, show_balance)
# - o clasa StreamingServicePaymentProxy (provider: str, subscription_price: int, subscription_expiration_date: datetime, account: BankAccount)
# care sa implementeze un design pattern potrivit si sa faca handling asupra abonamentului, permitand vizualizarea unui show
# numai in cazul in care abonamentul este inca activ sau, in caz contrar, daca in contul bancar mai sunt suficiente fonduri
# pentru a finanta inca o luna (caz in care se retrage suma din cont si se prelungeste abonamentul cu 30 de zile)
# - o clasa AuthorizedAccount (authorized_person: BankAccount, observable: BankAccount) care sa implementeze un design pattern potrivit pentru a desemna o persoana imputernicita
# asupra contului bancar, aceasta fiind notificata asupra oricarei operatiuni de retragere sau depunere de bani din cont cu 
# un mesaj sugestiv, de ex: ">Notification from account nr <....> (owner <....>): <mesajul tranzactiei>"
# Pasi sugerati pentru testarea programului: 
# 1) instantierea unui BankAccount (preferabil cu 0 lei)
# 2) instantierea unui BankAccount de imputernicit
# 3) instantierea unui AuthorizedAccount al imputernicit catre contul initial
# 4) instantierea unui StreamingServicePaymentProxy legat la primul cont (preferabil cu abonamentul expirat) 
# 5) apel catre watching_a_show(show_name) (daca abonamentul e expirat si contul nu are bani, ar trebui sa va afiseze un mesaj sugestiv)
# 6) depunere bani in cont (pe langa depunerea in sine, persoana imputernicita ar trebui sa primeasca un mesaj de notificare despre operatiune)
# 7) apel catre watching_a_show (pe langa mesajul despre show-ul urmarit, acum ar trebui sa primim un mesaj de succes cu privire la tranzactia din cont si unul cu privire la actualizarea abonamentului, cat si unul cu notificarea imputernicitului)

from datetime import datetime, timedelta


class StreamingService:
    def __init__(self, provider):
        self.provider = provider
    def watching_a_show(self, show):
        print(f'You are now watching {show} on {self.privider}')

class BankAccount:
    def __init__(self, owner, iban, balance = 0):
        self.owner = owner
        self.iban = iban
        self.balance = balance
        self._observers = []
    def __str__(self):
        return f'{self.owner} aveti in contul dumneavoastra {self.iban} suma de {self.balance} lei.'
    def show_balance(self):
        print(self.__str__())
    def withdraw(self, amount):
        if self.balance >= amount:
            self.balance -= amount
            message = f'Tranzactie reusita, {amount} a fost debitat din contul dvs.'
            print(message)
            self.notify_observers(message)
            return True
        
        else:
            message = f'Tranzactie nereusita, fonduri insuficiente.'
            print(message)
            self.notify_observers(message)
            return False
    
    def deposit(self, amount):
        self.balance += amount
        
        message = f'{amount} a fost depozitata contului dvs.'
        print(message)
        self.notify_observers(message)
        
    def register_observer(self, observer):
        self._observers.append(observer)
        
    def notify_observers(self, message):
        for obs in self._observers:
            obs.notify(self, message)

class StreamingServicePaymentProxy:
    def __init__(self, provider, subscription_price, expiration_date, account):
        self.streaming_service = StreamingService(provider)
        self.subscription_price = subscription_price
        self.subscription_expiration_date = datetime.strptime(expiration_date, '%Y-%m-%d')
        self.account = account
    def watching_a_show(self, show):
        if self.subscription_expiration_date > datetime.now():
            self.streaming_service.watching_a_show(show)
        else:
            if self.account.withdraw(self.subscription_price):
                self.subscription_expiration_date += timedelta(days = 30)
                print(f'Abonamentul dvs a fost reinoit, noua data de expirare va fi la {self.subscription_expiration_date}')
            else:
                print(f'Abonamentul dvs a expirat si nu aveti fonduri suficiente pentru a-l reinoi, ne pare rau!')

class AuthorisedAccount:
    def __init__(self, authorised_person, observable):
        self.authorised_person = authorised_person
        observable.register_observer(self)
    def notify(self, observable, message):
        print(f'Notificare de la contul nr. {observable.iban} detinator {observable.owner}: mesaj {message}')
        

florin = BankAccount('florin', 'ROBTRL4567890')
florin.show_balance()

streaming_service = StreamingServicePaymentProxy('Netflix', 40, '2022-3-4', florin)
streaming_service.watching_a_show('Outlander')

florin.deposit(3000)
florin.show_balance()

streaming_service.watching_a_show('Outlander')
                
        
        
        
            
        
            
    
        

florin aveti in contul dumneavoastra ROBTRL4567890 suma de 0 lei.
Tranzactie nereusita, fonduri insuficiente.
Abonamentul dvs a expirat si nu aveti fonduri suficiente pentru a-l reinoi, ne pare rau!
3000 a fost depozitata contului dvs.
florin aveti in contul dumneavoastra ROBTRL4567890 suma de 3000 lei.
Tranzactie reusita, 40 a fost debitat din contul dvs.
Abonamentul dvs a fost reinoit, noua data de expirare va fi la 2022-04-03 00:00:00
